### Recommendations with MovieTweetings: Getting to Know The Data

Throughout this lesson, you will be working with the [MovieTweetings Data](https://github.com/sidooms/MovieTweetings/tree/master/recsyschallenge2014).  To get started, you can read more about this project and the dataset from the [publication here](http://crowdrec2013.noahlab.com.hk/papers/crowdrec2013_Dooms.pdf).

**Note:** There are solutions to each of the notebooks available by hitting the orange jupyter logo in the top left of this notebook.  Additionally, you can watch me work through the solutions on the screencasts that follow each workbook. 

To get started, read in the libraries and the two datasets you will be using throughout the lesson using the code below.

 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tests as t

%matplotlib inline

# Read in the datasets
movies = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/movies.dat',
                     delimiter='::', header=None, names=['movie_id', 'movie', 'genre'],
                     dtype={'movie_id': object}, engine='python')
reviews = pd.read_csv('https://raw.githubusercontent.com/sidooms/MovieTweetings/master/latest/ratings.dat',
                      delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'],
                      dtype={'movie_id': object, 'user_id': object, 'timestamp': object}, engine='python')

#### 1. Take a Look At The Data 

Take a look at the data and use your findings to fill in the dictionary below with the correct responses to show your understanding of the data.

In [2]:
print('There are {} movies:\n'.format(movies.shape[0]))
movies.head()

There are 34583 movies:



,movie_id,movie,genre
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short
1,0000010,La sortie des usines Lumière (1895),Documentary|Short
2,0000012,The Arrival of a Train (1896),Documentary|Short
3,25,The Oxford and Cambridge University Boat Race ...,NaN
4,0000091,Le manoir du diable (1896),Short|Horror


In [3]:
print('There are {} reviews:\n'.format(reviews.shape[0]))
reviews.head()

There are 819671 reviews:



,user_id,movie_id,rating,timestamp
0,1,0111161,10,1373234211
1,1,0117060,7,1373415231
2,1,0120755,6,1373424360
3,1,0317919,6,1373495763
4,1,0454876,10,1373621125


In [4]:
genres = []

for genre in movies.genre:
    try:
        genres.extend(genre.split('|'))
    except AttributeError:
        pass

genres = set(genres)
print("The number of genres is {}:\n".format(len(genres)))
genres

The number of genres is 28:



{'Action',
 'Adult',
 'Adventure',
 'Animation',
 'Biography',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Film-Noir',
 'Game-Show',
 'History',
 'Horror',
 'Music',
 'Musical',
 'Mystery',
 'News',
 'Reality-TV',
 'Romance',
 'Sci-Fi',
 'Short',
 'Sport',
 'Talk-Show',
 'Thriller',
 'War',
 'Western'}

In [5]:
unique_users = reviews['user_id'].unique()
print('There are {} unique users.\n'.format(unique_users.size))

There are 60649 unique users.



In [6]:
print('There are {} missing ratings.'.format(reviews['rating'].isnull().sum()))

There are 0 missing ratings.


In [7]:
reviews.describe()

,rating
count,819671.000000
mean,7.305756
std,1.855686
min,0.000000
25%,6.000000
50%,7.000000
75%,9.000000
max,10.000000


In [8]:
# Use your findings to match each variable to the correct statement in the dictionary


dict_sol1 = {
'The number of movies in the dataset': 34574,
'The number of ratings in the dataset': 819467,
'The number of different genres': 28,
'The number of unique users in the dataset': 60636,
'The number missing ratings in the reviews dataset': 0,
'The average rating given across all ratings': 7.3,
'The minimum rating given across all ratings': 0,
'The maximum rating given across all ratings': 10
}

# Originally, I had this to check your solution, but the 
# links are live and updating.  That didn't end up being
# a great idea


#### 2. Data Cleaning

Next, we need to pull some additional relevant information out of the existing columns. 

For each of the datasets, there are a couple of cleaning steps we need to take care of:

#### Movies
* Pull the date from the title and create new column
* Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
* Dummy column the genre with 1's and 0's

#### Reviews
* Create a date out of time stamp

You can check your results against the header of my solution by running the cell below with the **show_clean_dataframes** function.

In [9]:
# Create the 'year' column by extracting it from the last parentheses in the movie name
# and keeping only the century

import re
import math

def extract_century(movie_name):
    year = int(re.search(r'\((.*?)\)', movie_name).group(1))
    return math.floor(year/100)*100

movies['century'] = movies['movie'].apply(extract_century)
movies.head()

,movie_id,movie,genre,century
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1800
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1800
2,0000012,The Arrival of a Train (1896),Documentary|Short,1800
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1800
4,0000091,Le manoir du diable (1896),Short|Horror,1800


In [10]:
# Dummy the 'century' feature

movies = pd.concat([movies, pd.get_dummies(movies['century'])], axis=1)
movies.drop(columns='century', inplace=True)
movies.head()

,movie_id,movie,genre,1800,1900,2000
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1,0,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1,0,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1,0,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1,0,0


In [11]:
# Dummy the 'genre' feature

def split_genres(genres):
    try:
        if genres.find(genre) > -1:    # any value >= 0 means the string was found
            return 1
        else:
            return 0
    except AttributeError:    # if the genre is NaN
        return 0

for genre in genres:
    movies[genre] = movies['genre'].apply(split_genres)
    
movies.head()

,movie_id,movie,genre,1800,1900,2000,Adventure,Film-Noir,Thriller,Romance,...,Biography,Sport,Reality-TV,History,Music,Animation,Mystery,War,Short,Comedy
0,0000008,Edison Kinetoscopic Record of a Sneeze (1894),Documentary|Short,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0000010,La sortie des usines Lumière (1895),Documentary|Short,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0000012,The Arrival of a Train (1896),Documentary|Short,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,25,The Oxford and Cambridge University Boat Race ...,NaN,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0000091,Le manoir du diable (1896),Short|Horror,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [12]:
# Replace the timestamp column with a date column

from datetime import datetime

def timestamp_to_date(timestamp):
    return datetime.fromtimestamp(int(timestamp))

reviews['date'] = reviews['timestamp'].apply(timestamp_to_date)
reviews.drop(columns='timestamp', inplace=True)
reviews.head()

,user_id,movie_id,rating,date
0,1,0111161,10,2013-07-07 21:56:51
1,1,0117060,7,2013-07-10 00:13:51
2,1,0120755,6,2013-07-10 02:46:00
3,1,0317919,6,2013-07-10 22:36:03
4,1,0454876,10,2013-07-12 09:25:25


In [13]:
reviews_new, movies_new = t.show_clean_dataframes()

   Unnamed: 0  user_id  movie_id  rating   timestamp                 date  \
0           0        1     68646      10  1381620027  2013-10-12 23:20:27   
1           1        1    113277      10  1379466669  2013-09-18 01:11:09   
2           2        2    422720       8  1412178746  2014-10-01 15:52:26   
3           3        2    454876       8  1394818630  2014-03-14 17:37:10   
4           4        2    790636       7  1389963947  2014-01-17 13:05:47   

   month_1  month_2  month_3  month_4    ...      month_9  month_10  month_11  \
0        0        0        0        0    ...            0         1         0   
1        0        0        0        0    ...            0         0         0   
2        0        0        0        0    ...            0         1         0   
3        0        0        0        0    ...            0         0         0   
4        0        0        0        0    ...            0         0         0   

   month_12  year_2013  year_2014  year_2015  year